In [27]:
import pandas as pd
from scipy.signal import find_peaks
import plotly.graph_objects as go
import numpy as np
import itertools
# from peakdetect import peakdetect
import os
import matplotlib.pyplot as plt
from collections import defaultdict

In [28]:
# storage_dict = {} # initalise dictonary to save keys = volts_strings and the corresponding_list

# 1: Data importing and functions

In [29]:
folders = [0.24,   11.00 , 13.00 , 15.00  ,17.25 , 19.25 , 3.25	,5.50 , 7.75,
0.25 ,  11.25  ,13.25,  15.25,  1.75  , 19.50 , 3.50,	5.75 , 8.00,
0.50 ,  11.50 , 13.50 , 15.50  ,17.50  ,19.75  ,3.75	,6.00 ,8.25,
0.75  , 11.75  ,13.75  ,15.75 , 17.75,  19.99 , 4.00,	6.25,  8.50,
1.00  , 12.00,  14.00 , 16.00 , 18.00 , 2.00  , 4.25,	6.50 , 8.75,
10.00,  12.25 , 14.25,  16.25,  18.25,  2.25,   4.50,	6.75 , 9.00,
10.25,  1.25  , 14.50,  16.50 , 18.50 , 2.50,   4.75,	7.00 , 9.25,
10.50 , 12.50 , 14.75,  16.75,  18.75 , 2.75,   5.00,	7.25  ,9.50,
10.75 , 12.75 , 1.50  , 17.00  ,19.00 , 3.00,   5.25,	7.50,  9.75]

In [30]:
folders = np.sort(folders)
folders

array([ 0.24,  0.25,  0.5 ,  0.75,  1.  ,  1.25,  1.5 ,  1.75,  2.  ,
        2.25,  2.5 ,  2.75,  3.  ,  3.25,  3.5 ,  3.75,  4.  ,  4.25,
        4.5 ,  4.75,  5.  ,  5.25,  5.5 ,  5.75,  6.  ,  6.25,  6.5 ,
        6.75,  7.  ,  7.25,  7.5 ,  7.75,  8.  ,  8.25,  8.5 ,  8.75,
        9.  ,  9.25,  9.5 ,  9.75, 10.  , 10.25, 10.5 , 10.75, 11.  ,
       11.25, 11.5 , 11.75, 12.  , 12.25, 12.5 , 12.75, 13.  , 13.25,
       13.5 , 13.75, 14.  , 14.25, 14.5 , 14.75, 15.  , 15.25, 15.5 ,
       15.75, 16.  , 16.25, 16.5 , 16.75, 17.  , 17.25, 17.5 , 17.75,
       18.  , 18.25, 18.5 , 18.75, 19.  , 19.25, 19.5 , 19.75, 19.99])

In [31]:
path = "/Users/vashisth/Desktop/PHY_243W/data_at_279.800kHz/"
file_path_ls = [path+'{0:.2f}/{1:.2f}_05.csv'.format(x,x)  for x in folders]
file_path_ls[28] = path+'7.00/7.00_005.csv'
file_path_ls[30] = path + '7.50/7.50_005.csv'
file_path_ls[0]

'/Users/vashisth/Desktop/PHY_243W/data_at_279.800kHz/0.24/0.24_05.csv'

In [32]:
file_path_ls_2 = [path+'{0:.2f}/{1:.2f}_06.csv'.format(x,x)  for x in folders]
file_path_ls_2[28] = path+'7.00/7.00_006.csv'
file_path_ls_2[30] = path + '7.50/7.50_006.csv'
file_path_ls_2[0]

'/Users/vashisth/Desktop/PHY_243W/data_at_279.800kHz/0.24/0.24_06.csv'

In [33]:
# create a list to hold the grouped elements

def threshold_group_maker(lst, threshold):

    groups = {}
    grouped_lst = []
    # sort the list in ascending order

    lst.sort()

    # create a variable to hold the current group of elements

    current_group = []

    # iterate over the sorted list

    for i in range(len(lst)):
    # if the current element is within the given threshold of the previous element, add it to the current group
        if i > 0 and abs(lst[i] - lst[i-1]) <= threshold:
         current_group.append(lst[i])
        # if the current element is not within the given threshold of the previous element,
        # add the current group to the list of grouped elements and start a new group
        else:
            grouped_lst.append(current_group)
            current_group = [lst[i]]
    # add the final group of elements to the list of grouped elements

    grouped_lst.append(current_group)

    # return the list of grouped elements
    grouped_lst = grouped_lst  [1:]
    for i in range( len(grouped_lst) ):
        groups[i] = grouped_lst[i]
    return groups


# example usage: group all elements in the list that are within 2 of each other

In [34]:
def get_peaks_singlefile(file_path, volt_value, difference):
    rlc_data = pd.read_csv(filepath_or_buffer =file_path) # first file
    if rlc_data.columns[0] =='Frequency': # had error from 16.50 onwards
        rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})
    time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
    time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float) # output
    time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float) # driving
    # return time_values, time_series_A, time_series_B
    
    indices = find_peaks(time_series_B,  prominence=0, distance=20)[0]
    positive_indices = indices[time_series_B[indices] > 0]
    indices = positive_indices

    peaks_channel_B =  [time_series_B[j] for j in indices] # these are the peaks from find_peaks functiom
    peaks_channel_A =  [time_series_A[j] for j in indices] # output voltage corresponding to peaks of driving voltage
    
    return peaks_channel_B, peaks_channel_A


In [35]:
def run_one(file_path, file_path_2, volt_value, difference):
 
  peaks_channel_B_1, peaks_channel_A_1 = get_peaks_singlefile(file_path, volt_value, difference)
  peaks_channel_B_2, peaks_channel_A_2 = get_peaks_singlefile(file_path_2, volt_value, difference)
  
  peaks_channel_A = peaks_channel_A_1 + peaks_channel_A_2
  peaks_channel_B = peaks_channel_B_1 + peaks_channel_B_2
  
  tuples_A_B = list( zip(peaks_channel_A, peaks_channel_B) )
  tuples_A_B = sorted(tuples_A_B, key=lambda x: x[0])

  dict_groups = threshold_group_maker(sorted( list( peaks_channel_A) ), threshold=difference )
  key_vals = list ( dict_groups.keys() )

  peaks_channel_A_unique = [ np.mean(dict_groups[j]) for j in key_vals ]

  storage_dict = {}
  volts_string = str(volt_value)
  
  storage_dict[volts_string] = peaks_channel_A_unique
  return storage_dict

In [36]:
# check = run_one(file_path_ls[0],folders[0], difference=0.5)
# check

In [39]:
def run_all(ls_path,ls_path_2, ls_volts, difference_list):
  
  store_all = {}
  for i in range(len(ls_path)):
    path = ls_path[i]
    volt_value = ls_volts[i] # volt remains the same for both .05 and .06
    if volt_value< 2.5:
      difference = difference_list[0]
    elif 2.5<= volt_value < 5.5:
      difference = difference_list[1]
    elif 5.5<= volt_value < 12.5:
      difference = difference_list[2]
    else:
      difference = difference_list[3]
    
    store_one = run_one(file_path = path,file_path_2 = ls_path_2[i], 
                        volt_value = volt_value, difference = difference)
    store_all.update(store_one)
  return store_all

# Bifurcation
- Plot for different thresholds (when we take two peaks as separate)
- this has an adaptive threshold given it goes haywire after 12 ish

** You can play around with this list. We will probably go with the [1, 2, 2, 3]**

In [40]:
diff_l = [ [0.5, 1, 2, 2], [ 1, 2, 2, 3], [1.5, 3,5,5] , [1,1,1,1]]
i=0
for dist_list in diff_l: 
    results = run_all(file_path_ls,file_path_ls_2, folders, difference_list=dist_list)
    # results
    keys, vals , vals_len= [], [], []
    for k in results.keys():
        keys.append(float(k))
        vals.append(results.get(k))
        vals_len.append(len(results.get(k)))
    
    plt_lsx = []
    for j in range(len(vals)):
        ls = [vals[j][i] for i in range(len(vals[j])) ]
        for k in range(len(ls)):
            plt_lsx.append([float(keys[j]), ls[k]])

    x_axis = [i[0] for i in plt_lsx]  
    y_axis = np.asarray([i[1] for i in plt_lsx] ) 

    trace1 = go.Scatter(
        x = x_axis,
        y = y_axis,
        mode='markers ',   # scatter mode (more in UG section 1)
        opacity = .6,
        marker = dict(size = 5, color = 'blue', line=dict(width=0.5))
        )
    data = [trace1]

    layout = go.Layout(
        autosize=False,
        width=750,
        height=500,

        xaxis= go.layout.XAxis(linecolor = 'black',
                            linewidth = 1,
                            mirror = True),

        yaxis= go.layout.YAxis(linecolor = 'black',
                            linewidth = 1,
                            mirror = True),

        margin=go.layout.Margin(
            l=50,
            r=50,
            b=100,
            t=100,
            pad = 4
        )
    )
    fig = go.Figure(data=data, layout=layout)
    fig.update_layout(title= 'threshold =' + str(dist_list) )
    fig.update_layout(yaxis_title="Output Voltage (mV)", xaxis_title="Driving Voltage (V)")
    i = i+1
    fig.write_image("Bifurcation{}.png".format(i))
    fig.show()
    

# 3: Getting other plots
    - x vs y
    - peaks
    - how do we find the number of peaks demonstrated

In [13]:
import plotly.express as px
import plotly.io as pio

pio.templates.default = "seaborn"

def get_plot(file_path, volt_value):
  rlc_data = pd.read_csv(filepath_or_buffer =file_path) # first file

  if rlc_data.columns[0] =='Frequency': # had error from 16.50 onwards
    rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})
  
  time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
  time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float) # output
  time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float) # driving
  
  df = {"Output": time_series_A, "Driving":time_series_B}
  fig = px.line(df,
    x="Output", y="Driving", title='Driving (V) vs Output (mV) Voltage (y vs x) for {} V'.format(volt_value))
  fig.update_layout(
    width=500,
    height=500,
  )  
  fig.update_xaxes(showline=True, linewidth=2, linecolor='black')
  fig.update_yaxes(showline=True, linewidth=2, linecolor='black')
  fig.write_image("DvsO{}.png".format(volt_value))
  fig.show()

In [137]:
asdf = 68

#for i in range (len(folders)):
#    asdf = i
get_plot(file_path_ls_2[asdf], folders[asdf])

In [138]:
get_plot(file_path_ls_2[10], folders[10]) #

In [139]:
get_plot(file_path_ls_2[20], folders[20]) #

In [140]:
get_plot(file_path_ls_2[40], folders[40]) #

## x vs y plots

In [78]:
volts = '5.50'
rlc_data_1 = pd.read_csv(filepath_or_buffer = path+ (volts)+'/' + (volts)+'_05.csv') # first file
# rlc_data_2 = pd.read_csv(filepath_or_buffer = path+volts+'/' + volts+'_05.csv') # second file
# rlc_data_2 = rlc_data_2.drop(axis=0, index=0)

In [79]:
# rlc_data = rlc_data_1.append(rlc_data_2, ignore_index=True)
rlc_data = rlc_data_1
if rlc_data.columns[0] =='Frequency':
  rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})

In [80]:
time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float)
time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float)

In [81]:
## Sanity check
# fig = go.Figure(data=go.Scatter(
#     x = time_values,
#     y = time_series_B,
#     mode = 'markers'
# ))

# fig.show()

### Find and draw peaks

In [82]:
# indices = find_peaks(time_series_B,  prominence=0, distance=25, threshold=0.05 * 10**(-3))[0]
indices = find_peaks(time_series_B,  prominence=0, distance=20)[0]
positive_indices = indices[time_series_B[indices] > 0]
indices = positive_indices
indices

array([  25,  166,  310,  453,  597,  738,  881, 1025, 1166, 1310, 1453,
       1597, 1738, 1882])

In [89]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series_B,
    mode='lines+markers',
    name='Driving Voltage (V) '
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_B[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks (A)'
))
fig.update_layout(yaxis_title="Driving Voltage (V)", xaxis_title="")
fig.update_layout(title= volts )
fig.write_image("DvsO{}.png".format(volts))
fig.show()

## Drawing peaks of A and corresponding values of channel B

In [90]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    y=time_series_A,
    mode='lines+markers',
    name='Channel A (mV)'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_A[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='cross'
    ),
    name='Detected Peaks (A)'
))

fig.add_trace(go.Scatter(
    y=time_series_B,
    mode='lines+markers',
    name='Channel B (V)'
))

fig.add_trace(go.Scatter(
    x=indices,
    y=[time_series_B[j] for j in indices],
    mode='markers',
    marker=dict(
        size=8,
        color='red',
        symbol='circle'
    ),
    name='B values corr to Peak A'
))

fig.update_layout(yaxis_title="Voltage", xaxis_title=" index")
fig.update_layout(title= volts + " Volts" )
fig.write_image("ChaA_ChanB{}.png".format(volts))

fig.show()

In [93]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x = time_values, 
    y=time_series_A,
    mode='lines+markers',
    name='Channel A (mV)'
))



fig.add_trace(go.Scatter(
    x = time_values,
    y=time_series_B,
    mode='lines+markers',
    name='Channel B (V)'
))



fig.update_layout(yaxis_title="Voltage", xaxis_title=" index")
fig.update_layout(title= volts + " Volts" )
fig.write_image("ChaA_ChanB{}.png".format(volts))

fig.show()

In [55]:
peaks_channel_A =  [time_series_A[j] for j in indices]
peaks_channel_B =  [time_series_B[j] for j in indices]

In [56]:
tuples_A_B = list( zip(peaks_channel_A, peaks_channel_B) )
tuples_A_B = sorted(tuples_A_B, key=lambda x: x[0])
tuples_A_B

[(4.151738, 2.621693),
 (4.596697, 2.622303),
 (4.649799, 2.627796),
 (4.741965, 2.626881),
 (4.884181, 2.628407),
 (4.982451, 2.627033),
 (4.984282, 2.627644),
 (5.006256, 2.626575),
 (5.365764, 2.625355),
 (5.500045, 2.626118),
 (5.68926, 2.626728),
 (5.971862, 2.621998),
 (6.010925, 2.626423),
 (6.252632, 2.621082)]

# Phase plots fn

In [160]:
def phase_plot(volts, scaling = 1):
    # volts = '5.50'
    rlc_data_1 = pd.read_csv(filepath_or_buffer = path+ (volts)+'/' + (volts)+'_05.csv') # first file
    rlc_data = rlc_data_1
    
    if rlc_data.columns[0] =='Frequency':
        rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})
    
    time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
    time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float)
    time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float)
    time_series_A = np.asarray(time_series_A) * scaling
    
    indices = find_peaks(time_series_B,  prominence=0, distance=20)[0]
    indices = indices[time_series_B[indices] > 0]
    
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x = time_values,
        y=time_series_A,
        mode='lines+markers',
        name='Channel A (mV)'
    ))

    # fig.add_trace(go.Scatter(
    #     x=indices,
    #     y=[time_series_A[j] for j in indices],
    #     mode='markers',
    #     marker=dict(
    #         size=8,
    #         color='red',
    #         symbol='cross'
    #     ),
    #     name='Detected Peaks (A)'
    # ))

    fig.add_trace(go.Scatter(
        x = time_values,
        y=time_series_B,
        mode='lines+markers',
        name='Channel B (V)'
    ))

    # fig.add_trace(go.Scatter(
    #     x=indices,
    #     y=[time_series_B[j] for j in indices],
    #     mode='markers',
    #     marker=dict(
    #         size=8,
    #         color='red',
    #         symbol='circle'
    #     ),
    #     name='B values corr to Peak A'
    # ))

    fig.update_layout(yaxis_title="Voltage", xaxis_title=" Time (micro s)")
    fig.update_layout(title= volts + " Volts" )
    fig.write_image("ChaA_ChanB{}_noscatter.png".format(volts))
    fig.show()

In [161]:
phase_plot('17.00')

In [119]:
# def phase_plot(volts, scaling = 1):
#     # volts = '5.50'
#     rlc_data_1 = pd.read_csv(filepath_or_buffer = path+ (volts)+'/' + (volts)+'_05.csv') # first file
#     rlc_data = rlc_data_1
    
#     if rlc_data.columns[0] =='Frequency':
#         rlc_data = rlc_data.rename(columns = {'Frequency': 'Time'})
    
#     time_values = rlc_data ['Time'].to_numpy()  [1:].astype(float)
#     time_series_A = rlc_data['Channel A'].to_numpy() [1:].astype(float)
#     time_series_B = rlc_data['Channel B'].to_numpy() [1:].astype(float)
#     time_series_A = np.asarray(time_series_A) * scaling
    
#     indices = find_peaks(time_series_B,  prominence=0, distance=20)[0]
#     indices = indices[time_series_B[indices] > 0]
   
#     plt.figure(figsize=(10,6))
#     plt.plot(time_values, time_series_A, label = 'Channel A')
#     plt.plot(time_values, time_series_B, label = 'Channel B')
#     plt.legend()
#     plt.show()
    